# Lesson 9: Python Database Integration

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-8  
**Learning Mode:** Read explanations, then run each Python code cell

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Connect to SQLite databases from Python
- Execute SQL queries programmatically
- Fetch and process query results
- Use parameterised queries to prevent SQL injection
- Handle database errors gracefully
- Close connections properly
- Build simple database-driven applications


## 📚 Why Use Python with Databases?

Writing SQL queries in `.sql` files is great for learning, but real applications need to:

- ✅ Accept user input dynamically
- ✅ Process query results with programming logic
- ✅ Display results in custom formats
- ✅ Integrate databases with web apps, APIs, GUIs
- ✅ Automate database operations
- ✅ Handle errors and edge cases

**Python's `sqlite3` module** (built-in!) lets you do all of this.

### Real-World Use Cases:

- **Web Applications:** Store user accounts, posts, products
- **Data Analysis:** Query databases, process results with pandas
- **Automation Scripts:** Backup data, generate reports
- **Desktop Apps:** Save application settings, user data
- **APIs:** Serve database content as JSON


## 🔧 Part 1: Connecting to a Database

### Import the sqlite3 Module

**Note:** This notebook uses standard Python (not `%%sql` magic from previous lessons).


In [ ]:
import sqlite3
from pathlib import Path
import sys

print(f"Python version: {sys.version}")
print(f"sqlite3 version: {sqlite3.version}")


### Create a Database Connection

**Function to connect to our Star Wars database:**


In [ ]:
# We no longer need a create_connection function when using context managers!
# Context managers handle connection opening and closing automatically.

# For quick database info check:
def get_database_info(db_file='database/starwars.db'):
    """
    Display database connection information
    
    Args:
        db_file: Path to database file
    """
    try:
        with sqlite3.connect(db_file) as conn:
            print(f"✓ Connected to database: {db_file}")
            print(f"✓ SQLite version: {sqlite3.sqlite_version}")
            # Connection automatically closes when exiting the 'with' block
    except sqlite3.Error as e:
        print(f"✗ Error connecting to database: {e}")


### Test the Connection

**Run this cell to verify the database connection works:**


In [ ]:
# Test connection using context manager
get_database_info()
print("✓ Connection automatically closed")


**Important Concepts:**

- `sqlite3.connect(filename)` - Creates/opens a database file
- `with sqlite3.connect(db) as conn:` - Context manager automatically closes connection
- Context managers handle cleanup even if errors occur

**Best Practice:** Always use `with` statements (context managers) for database connections.


## 📊 Part 2: Executing Queries

### Create a Cursor

A **cursor** is like a pointer that executes SQL and fetches results.


In [ ]:
def get_all_characters():
    """Retrieve all characters from the database"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            # Create cursor
            cursor = conn.cursor()
            
            # Execute SQL query
            cursor.execute("SELECT id, name, species, homeworld FROM characters")
            
            # Fetch all results
            characters = cursor.fetchall()
            
            # Display results
            print("\n=== All Characters ===")
            for char in characters:
                print(f"ID: {char[0]}, Name: {char[1]}, Species: {char[2]}, Homeworld: {char[3]}")
            
            print(f"\nTotal characters: {len(characters)}")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Error executing query: {e}")

# Run the function
get_all_characters()


**Key Methods:**

| Method | Purpose | Returns |
|--------|---------|---------|
| `cursor.execute(sql)` | Run SQL query | Cursor object |
| `cursor.fetchall()` | Get all rows | List of tuples |
| `cursor.fetchone()` | Get one row | Tuple or None |
| `cursor.fetchmany(n)` | Get n rows | List of tuples |

**Each tuple represents one row** - access values by index.


### Different Fetch Methods

**Three ways to retrieve query results:**


In [ ]:
def demonstrate_fetch_methods():
    """Show fetchone(), fetchmany(), and fetchall()"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            # Method 1: fetchone() - Get one row at a time
            print("\n=== fetchone() Demo ===")
            cursor.execute("SELECT name, species FROM characters LIMIT 3")
            row = cursor.fetchone()
            while row:
                print(f"Name: {row[0]}, Species: {row[1]}")
                row = cursor.fetchone()
            
            # Method 2: fetchmany() - Get specified number of rows
            print("\n=== fetchmany() Demo ===")
            cursor.execute("SELECT name, species FROM characters")
            rows = cursor.fetchmany(5)  # Fetch 5 rows
            for row in rows:
                print(f"Name: {row[0]}, Species: {row[1]}")
            
            # Method 3: fetchall() - Get all remaining rows
            print("\n=== fetchall() Demo ===")
            cursor.execute("SELECT name, height FROM characters WHERE height IS NOT NULL ORDER BY height DESC LIMIT 3")
            all_rows = cursor.fetchall()
            print("Top 3 Tallest Characters:")
            for row in all_rows:
                print(f"{row[0]}: {row[1]}cm")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

demonstrate_fetch_methods()


**When to use each:**

- `fetchone()` - Process one row at a time (memory efficient for huge results)
- `fetchmany(n)` - Process in batches (balance between memory and speed)
- `fetchall()` - Get everything at once (convenient for small result sets)


### Using Row Factory for Named Columns

**Problem:** `row[0]`, `row[1]` is hard to read!

**Solution:** `Row` objects act like dictionaries:


In [ ]:
def get_characters_as_dict():
    """Get results as dictionary-like objects"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            # Set row factory to return Row objects
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT name, species, height 
                FROM characters 
                WHERE height IS NOT NULL 
                ORDER BY height DESC 
                LIMIT 5
            """)
            
            characters = cursor.fetchall()
            
            print("\n=== Characters (Row objects) ===")
            for char in characters:
                # Access by column name!
                print(f"{char['name']}: {char['height']}cm, {char['species']}")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

get_characters_as_dict()


**Much better!** `char['name']` is clearer than `char[0]`.

**Tip:** Always use `conn.row_factory = sqlite3.Row` for readable code.


## 🛡️ Part 3: Parameterised Queries (SQL Injection Prevention)

### The Danger of String Formatting

**NEVER do this:**

```python
# ❌ DANGEROUS - Vulnerable to SQL injection!
species = input("Enter species: ")
query = f"SELECT * FROM characters WHERE species = '{species}'"
cursor.execute(query)
```

**Why?** A malicious user could enter: `' OR '1'='1`  
**Result:** `SELECT * FROM characters WHERE species = '' OR '1'='1'`  
**Effect:** Returns ALL characters (security breach!)

### Safe Approach: Parameterised Queries

**Use placeholders (`?`) instead:**


In [ ]:
def safe_query_by_species(species):
    """Safe parameterised query"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            # Use ? placeholder - SQLite escapes input automatically
            cursor.execute("""
                SELECT name, species, homeworld 
                FROM characters 
                WHERE species = ?
            """, (species,))  # Note: tuple with comma
            
            results = cursor.fetchall()
            
            print(f"\n=== Characters of species: {species} ===")
            for char in results:
                print(f"- {char['name']} from {char['homeworld']}")
            print(f"Found: {len(results)} characters")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

# Test with different species
safe_query_by_species('Human')
safe_query_by_species('Droid')


**Key Points:**

- `?` is a placeholder for a value
- Pass values as a tuple: `(value,)` or `(val1, val2)`
- SQLite automatically escapes special characters
- **Always use parameterised queries for user input!**

### Multiple Parameters


In [ ]:
def query_by_species_and_height(species, min_height):
    """Query with multiple parameters"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            # Multiple placeholders
            cursor.execute("""
                SELECT name, species, height 
                FROM characters 
                WHERE species = ? AND height >= ?
                ORDER BY height DESC
            """, (species, min_height))
            
            results = cursor.fetchall()
            
            print(f"\n=== {species}s taller than {min_height}cm ===")
            for char in results:
                print(f"- {char['name']}: {char['height']}cm")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

query_by_species_and_height('Human', 170)


## 🔄 Part 4: Inserting, Updating, and Deleting Data

### Insert Data

**Add a new character:**


In [ ]:
def insert_character(name, species, homeworld):
    """Insert a new character"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                INSERT INTO characters (name, species, homeworld)
                VALUES (?, ?, ?)
            """, (name, species, homeworld))
            
            # IMPORTANT: Commit happens automatically with context manager on success!
            print(f"✓ Inserted: {name} (ID: {cursor.lastrowid})")
            # Connection automatically closes and commits here
            
    except sqlite3.Error as e:
        print(f"✗ Error inserting character: {e}")
        # Rollback happens automatically with context manager on error

# Insert a test character
insert_character('Test Character', 'Test Species', 'Test Planet')


**Critical:** With context managers, `conn.commit()` happens automatically on success!

**On error:** Context manager automatically rolls back changes.

### Update Data


In [ ]:
def update_character_species(name, new_species):
    """Update a character's species"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                UPDATE characters 
                SET species = ? 
                WHERE name = ?
            """, (new_species, name))
            
            # Commit happens automatically with context manager
            print(f"✓ Updated {cursor.rowcount} row(s)")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Error updating: {e}")
        # Rollback happens automatically

# Update the test character
update_character_species('Test Character', 'Updated Species')


**`cursor.rowcount`** tells you how many rows were affected.

### Delete Data


In [ ]:
def delete_character(name):
    """Delete a character by name"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                DELETE FROM characters 
                WHERE name = ?
            """, (name,))
            
            # Commit happens automatically with context manager
            if cursor.rowcount > 0:
                print(f"✓ Deleted {cursor.rowcount} character(s)")
            else:
                print(f"⚠ No character found with name: {name}")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Error deleting: {e}")
        # Rollback happens automatically

# Delete the test character
delete_character('Test Character')


## 🎯 Part 5: Practical Application

Let's build a simple character search function:


In [ ]:
def search_characters(search_term):
    """Search characters by name (case-insensitive)"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            # Use LIKE with wildcards
            cursor.execute("""
                SELECT name, species, homeworld, height
                FROM characters
                WHERE name LIKE ?
                ORDER BY name
            """, (f'%{search_term}%',))
            
            results = cursor.fetchall()
            return results
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")
        return []

# Test the search
print("\n=== Search Results ===")
results = search_characters('Luke')
for char in results:
    print(f"- {char['name']} ({char['species']}) from {char['homeworld']}")

results = search_characters('a')  # Find all with 'a' in name
print(f"\nFound {len(results)} characters with 'a' in their name")


### Build a Character Statistics Function


In [ ]:
def get_character_statistics():
    """Get various statistics about characters"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            # Total count
            cursor.execute("SELECT COUNT(*) FROM characters")
            total = cursor.fetchone()[0]
            
            # Count by species
            cursor.execute("""
                SELECT species, COUNT(*) as count
                FROM characters
                GROUP BY species
                ORDER BY count DESC
            """)
            species_counts = cursor.fetchall()
            
            # Average height
            cursor.execute("SELECT AVG(height) FROM characters WHERE height IS NOT NULL")
            avg_height = cursor.fetchone()[0]
            
            # Tallest and shortest
            cursor.execute("""
                SELECT name, height 
                FROM characters 
                WHERE height IS NOT NULL 
                ORDER BY height DESC 
                LIMIT 1
            """)
            tallest = cursor.fetchone()
            
            cursor.execute("""
                SELECT name, height 
                FROM characters 
                WHERE height IS NOT NULL 
                ORDER BY height ASC 
                LIMIT 1
            """)
            shortest = cursor.fetchone()
            
            # Display results
            print("\n=== Character Statistics ===")
            print(f"Total characters: {total}")
            print(f"Average height: {avg_height:.1f}cm")
            print(f"Tallest: {tallest[0]} ({tallest[1]}cm)")
            print(f"Shortest: {shortest[0]} ({shortest[1]}cm)")
            print(f"\nCharacters by species:")
            for species, count in species_counts:
                print(f"  {species}: {count}")
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

get_character_statistics()


## 🔒 Part 6: Error Handling and Best Practices

### Use Context Managers (with statement)

**Best practice:** Use `with` to auto-close connections:


In [ ]:
def get_characters_safe():
    """Using context manager for automatic cleanup"""
    try:
        # Connection automatically closes after 'with' block
        with sqlite3.connect('database/starwars.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            cursor.execute("SELECT name, species FROM characters LIMIT 5")
            characters = cursor.fetchall()
            
            for char in characters:
                print(f"- {char['name']} ({char['species']})")
            
        # Connection is now closed automatically
        print("✓ Connection auto-closed")
        
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")

get_characters_safe()


**Benefits:**
- Connection closes automatically (even if error occurs)
- Changes commit automatically on success
- Rollback happens automatically on errors
- Cleaner, more Pythonic code
- No need to track connection lifecycle manually

### Transaction Management


In [ ]:
def transfer_character_safe(name, new_homeworld):
    """Update with transaction management"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            # Start transaction (implicit with 'with')
            cursor.execute("""
                UPDATE characters 
                SET homeworld = ? 
                WHERE name = ?
            """, (new_homeworld, name))
            
            if cursor.rowcount == 0:
                raise ValueError(f"Character '{name}' not found")
            
            # Commit happens automatically if no exception
            print(f"✓ Transferred {name} to {new_homeworld}")
            
    except ValueError as e:
        print(f"⚠ {e}")
    except sqlite3.Error as e:
        print(f"✗ Database error: {e}")
        # Rollback happens automatically

# Test (won't actually change anything if name doesn't exist)
transfer_character_safe('NonExistent', 'Nowhere')

## 🎓 Practice Exercises

### Exercise 1: Create a Function

**Task:** Write a function that returns all characters from a specific planet.


In [ ]:
def get_characters_from_planet(planet_name):
    """Get all characters from a specific planet"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT c.name, c.species 
                FROM characters c
                WHERE c.homeworld = ?
            """, (planet_name,))
            
            results = cursor.fetchall()
            
            print(f"\nCharacters from {planet_name}:")
            for char in results:
                print(f"- {char['name']} ({char['species']})")
            
            return results
            # Connection automatically closes here
            
    except sqlite3.Error as e:
        print(f"Error: {e}")
        return []

# Test it
get_characters_from_planet('Tatooine')

### Exercise 2: Count Characters by Attribute

**Task:** Count characters grouped by any attribute:


In [ ]:
def count_by_attribute(attribute):
    """Count characters grouped by specified attribute"""
    try:
        with sqlite3.connect('database/starwars.db') as conn:
            cursor = conn.cursor()
            
            # Build query dynamically (CAREFUL - validate attribute name!)
            valid_attributes = ['species', 'homeworld', 'affiliation']
            if attribute not in valid_attributes:
                print(f"Invalid attribute. Choose from: {valid_attributes}")
                return
            
            query = f"""
                SELECT {attribute}, COUNT(*) as count
                FROM characters
                WHERE {attribute} IS NOT NULL
                GROUP BY {attribute}
                ORDER BY count DESC
            """
            
            cursor.execute(query)
            results = cursor.fetchall()
            
            print(f"\nCharacter count by {attribute}:")
            for row in results:
                print(f"  {row[0]}: {row[1]}")
                
    except sqlite3.Error as e:
        print(f"Error: {e}")

count_by_attribute('species')

## ✅ Checkpoint & Summary

### What You've Built

- ✅ Connect to SQLite databases from Python
- ✅ Execute parameterised queries to prevent SQL injection
- ✅ Fetch results with `fetchone()` and `fetchall()`
- ✅ Manage transactions with commit/rollback
- ✅ Use context managers (`with`) for clean resource handling
- ✅ Handle errors gracefully with try/except
- ✅ Build reusable helper functions for database access
- ✅ Integrate SQL queries into Python applications

### Key Python sqlite3 Functions

| Function/Method | Purpose |
|-----------------|---------|
| `sqlite3.connect(db)` | Create database connection |
| `conn.cursor()` | Create cursor for executing SQL |
| `cursor.execute(sql, params)` | Execute SQL query |
| `cursor.fetchone()` | Fetch one row |
| `cursor.fetchall()` | Fetch all rows |
| `conn.commit()` | Save changes permanently |
| `conn.rollback()` | Undo uncommitted changes |
| `conn.close()` | Close connection |
| `conn.row_factory = sqlite3.Row` | Enable named column access |
| `cursor.lastrowid` | Get ID of last inserted row |
| `cursor.rowcount` | Get number of affected rows |

### Best Practices Checklist

- [ ] Always use parameterised queries (`?` placeholders)
- [ ] Commit changes with `conn.commit()`
- [ ] Close connections when done
- [ ] Use `try/except` for error handling
- [ ] Use `with` statements for automatic cleanup
- [ ] Use `conn.row_factory = sqlite3.Row` for readability
- [ ] Validate input before building dynamic queries
- [ ] Never use f-strings or string formatting with user input

## 🎉 Congratulations!

You can now build Python applications that interact with databases! This is a fundamental skill for:
- Web development (Flask, Django, FastAPI)
- Data analysis and reporting
- Automation scripts
- Desktop applications
- And much more!

**Ready for the final lesson?** Open `lesson10_comparison.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson9_database.ipynb database/starwars.db
git commit -m "Completed Lesson 9: Python database integration"
git push
```
